In [ ]:
file_config = {"preamble_len": 4,
                "preamble_text": "\documentclass{article} \n\usepackage{polski} \n\usepackage[utf8]{inputenc} \n\begin{document}"}
tasks_config = {"Dynamika": {"Równia pochyła": [{"treść": "AAA\nBBB", "linie": (7,8)}]}}

In [2]:
class TasksBaseManager(object):
    
    def __init__(self):
        file_config = {"preamble_len": 4,
                       "preamble_text": "\documentclass{article} \n\usepackage{polski} \n\usepackage[utf8]{inputenc} \n\begin{document}",
                       "end_text": " \n \end{document}"}
        tasks_config = [{"Dynamika": [{"Równia pochyła": [{"treść": "AAA\nBBB", "linie": (7,8)}]}]}]
        
    def write_tasks_to_tex(self):
        tex_file = self.file_config["preamble_text"]
        for section, section_tasks in self.tasks_config.items():
            tex_file += " \n section\{{0}\}".format(section)
            for subsection, subsection_tasks in section_tasks.items():
                tex_file += " \n subsection\{{0}\}".format(subsection)
                for task in subsection_tasks:
                    tex_file += task["treść"]
        tex_file += self.file_config["end_text"]
        return tex_file

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 26-27: truncated \uXXXX escape (<ipython-input-2-1ad3a9ba5ede>, line 5)

In [1]:
tm = TasksBaseManager()
tm.write_tasks_to_tex()

NameError: name 'TasksBaseManager' is not defined